# Notebook 02: Clustering aglomerativo

En este notebook se va a implementar el algoritmo de clustering aglomerativo con centroid-linkage. En la primera parte se explica la implementación manual del algoritmo y luego se muestra como realizar los cálculos de forma sencilla con la librería scipy.cluster.

In [ ]:
import numpy as np
import tensorflow as tf # Solamente lo utilizamos para descargar los datos
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

Primero generamos los puntos aleatorios según tres distribuciones normales.

In [ ]:
d1 = np.random.randn(20, 2) + 2
d2 = np.random.randn(20, 2) - 3
d3 = np.random.randn(20, 2)
d3[:, 0] = d3[:, 0] + 4
d3[:, 1] = d3[:, 1] - 4

In [ ]:
plt.plot(d1[:, 0], d1[:, 1], '.')
plt.plot(d2[:, 0], d2[:, 1], '.')
plt.plot(d3[:, 0], d3[:, 1], '.')
plt.show()

Nuestro conjunto de datos va a ser la concatenación de las tres distribuciones.

In [ ]:
datos = np.concatenate((d1, d2, d3), axis=0)
datos = datos[np.random.permutation(len(datos))]

Se muestra la matriz de distancias para comprobar que los datos se han mezclado correctamente.

In [ ]:
matriz_distancias = euclidean_distances(datos)
plt.figure(figsize=(10, 10))
plt.imshow(matriz_distancias)
plt.colorbar()
plt.show()

## Implementación manual del algoritmo

Vamos a implementar el Centroid-Linkage a mano:

In [ ]:
K = 3

In [ ]:
# En cada ejecución de esta celda los datos se vuelven a concatenar de forma aleatoria
datos = np.concatenate((d1, d2, d3), axis=0)
datos = datos[np.random.permutation(len(datos))]
# Se guarda una copia de los datos originales para calcular los centroides
datos_original = np.copy(datos)

# TO-DO A partir de aquí comienza el algoritmo
# Se guardan los singleton clusters para calcular los centroides
set_lists = ??

while ??:
  # Se calcula la matriz de distancias

  # OJO: Importante "distorsionar" la diagonal para no considerar las distancias 0s con el mínimo

  # Obtenemos el índice del argmin
  ind = np.unravel_index(np.argmin(matriz_distancias), matriz_distancias.shape)

  # Actualización de la lista de clusters.

  set_lists.remove(set1)
  set_lists.remove(set2)
  set_lists.append(nuevo_set)

  # Actualización de la lista de centroides con el nuevo centroide
  nuevo = datos_original[nuevo_set].mean(axis=0, keepdims=True)
  datos = np.delete(datos, ind, 0)
  datos = np.append(datos, nuevo, axis=0)
  print(np.around(matriz_distancias.min(), decimals=4), "\t", set_lists)

In [ ]:
for i, set_list in enumerate(set_lists):
  plt.plot(datos_original[set_list, 0], datos_original[set_list, 1], '.', label="Cluster "+str(i+1))
plt.legend()
plt.show()

## Usando la librería scipy

https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html

In [ ]:
# En cada ejecución de esta celda los datos se vuelven a concatenar de forma aleatoria
datos = np.concatenate((d1, d2, d3), axis=0)
datos = datos[np.random.permutation(len(datos))]
# En particular, hemos implementado el linkage centroid (se indica con el parámetro method)
# con la distancia euclidea (parámetro metric)
Z = linkage(datos, method='centroid', metric='euclidean')

Manualmente, dibujar el dendrograma resulta algo tedioso. Sin embargo, la librería scipy está preparada para dibujar directamente el dendrograma si lo necesitamos:

In [ ]:
plt.figure(figsize=(20, 6))
dn = dendrogram(Z)
plt.grid()
plt.show()

Para poder separar los datos en los clusters que queremos, solamente necesitamos realizar el "corte" al dendrograma:

In [ ]:
fcluster(Z, 3.74, criterion='distance')

In [ ]:
indices = fcluster(Z, 5.08, criterion='distance')
for i in np.unique(indices):
  plt.plot(datos[indices == i, 0], datos[indices == i, 1], '.', label="Cluster "+str(i))
plt.legend()
plt.show()

## Clustering Aglomerativo para los datos de MNIST

Vamos a descargar el dataset de MNIST para hacer clustering con los datos de MNIST:

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Dividimos entre 255 para que todos los atributos de la imagen estén entre 0 y 1.

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

Restamos ahora la media para destacar como positivos los píxeles de la clase y el resto con valores negativos.

In [ ]:
x_train = x_train - (x_train.mean(axis=0) + 1e-16)
x_test = x_test - (x_train.mean(axis=0) + 1e-16)

Veamos el aspecto que tienen nuestros datos:

In [ ]:
plt.figure(figsize=(15,4))
for i in range(20):
  plt.subplot(2,10,i+1)
  plt.imshow(x_train[np.random.randint(60000)], cmap="bwr", vmin=-1, vmax=1)
plt.show()

In [ ]:
datos = x_train.reshape(x_train.shape[0], -1)
datos.shape

(60000, 784)

Para agilizar los cálculos (60000 imágenes tarda un buen rato) vamos a reducir el número de imágenes a 2000.

In [ ]:
datos = datos[:10000]
datos.shape

(10000, 784)

In [ ]:
Z = linkage(datos, method='single', metric='euclidean')

In [ ]:
plt.figure(figsize=(20, 6))
dn = dendrogram(Z, truncate_mode="level", p=20)
plt.grid()
plt.show()

In [ ]:
#clusters = fcluster(Z, 5.0, criterion='distance') # CENTROID
clusters = fcluster(Z, 5.0, criterion='distance') # SINGLE
len(np.unique(clusters))

In [ ]:
for c in np.unique(clusters):
  indices = np.where(clusters == c)[0]
  if len(indices) > 10:
    plt.figure(figsize=(15,4))
    plt.title("Cluster " + str(c))
    for i in range(10):
      plt.subplot(1,10,i+1)
      plt.imshow(x_train[indices][i], cmap="bwr", vmin=-1, vmax=1)
    plt.show()

## Conclusión

* Hemos comprendido el funcionamiento del clustering Aglomerativo.

* Hemos implementado el Centroid-Linkage a mano.

* Cuando usamos Clustering, **no tenemos las mismas herramientas que cuando nos enfrentamos a un problema de clasificación**. Dos clusters distintos pueden pertenecer a la misma clase, ya que representan patrones distintos.

* Diferentes estrategias de Linkage dan resultados distintos.